In [1]:
import json
from datasets import load_dataset
import numpy as np

dataset_id = "deepmind/narrativeqa"

dataset = load_dataset(dataset_id, split="test")
dataset

c:\Users\vohun\anaconda3\envs\py10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 24/24 [00:02<00:00, 11.32it/s]


Dataset({
    features: ['document', 'question', 'answers'],
    num_rows: 10557
})

In [2]:
def get_context(ex):
  ctx = ex["document"]["summary"]["text"]
  ctx_id = ex["document"]["id"]
  question = ex["question"]["text"]
  answers = [each["text"] for each in ex["answers"]]
  return dict(context=ctx, context_id=ctx_id, question=question, answers=answers)
dataset = dataset.map(get_context)
dataset = dataset.remove_columns(["document"])
dataset[1]
print(dataset)
col = dataset[0]
print(col)

Dataset({
    features: ['question', 'answers', 'context', 'context_id'],
    num_rows: 10557
})
{'question': 'Who is Mark Hunter?', 'answers': ['He is a high school student in Phoenix.', 'A loner and outsider student with a radio station.'], 'context': ' Mark Hunter (Slater), a high school student in a sleepy suburb of Phoenix, Arizona, starts an FM pirate radio station that broadcasts from the basement of his parents\' house. Mark is a loner, an outsider, whose only outlet for his teenage angst and aggression is his unauthorized radio station. His pirate station\'s theme song is "Everybody Knows" by Leonard Cohen and there are glimpses of cassettes by such alternative musicians as The Jesus and Mary Chain, Camper Van Beethoven, Primal Scream, Soundgarden, Ice-T, Bad Brains, Concrete Blonde, Henry Rollins, and The Pixies. By day, Mark is seen as a loner, hardly talking to anyone around him; by night, he expresses his outsider views about what is wrong with American society. When he sp

In [3]:
from collections import Counter

unique_contexts = Counter(dataset["context"])
count = 0
raft_train_ctx = []
raft_test_ctx = []
for k, v in unique_contexts.items():
  if v > 5 and count < 50:
    count += 1
    raft_train_ctx.append(k)
  else:
    raft_test_ctx.append(k)

print(len(raft_test_ctx), len(raft_train_ctx))

#raft_train_ctx = unique_contexts[:100]
#raft_test_ctx = unique_contexts[100:]

305 50


In [4]:
test_dataset = dataset.filter(lambda x: x["context"] in raft_test_ctx)
test_dataset

Filter: 100%|██████████| 10557/10557 [00:00<00:00, 61128.98 examples/s]


Dataset({
    features: ['question', 'answers', 'context', 'context_id'],
    num_rows: 9059
})

In [6]:
from datasets import Dataset

test_dataset_unique_ctx = test_dataset.to_pandas()
test_dataset_unique_ctx = test_dataset_unique_ctx.groupby("context").first().reset_index()
test_dataset_unique_ctx = Dataset.from_pandas(test_dataset_unique_ctx)
test_dataset_unique_ctx

Dataset({
    features: ['context', 'question', 'answers', 'context_id'],
    num_rows: 305
})

In [8]:
# Dataset for evaluation raft
uploaded_dataset_id = "phatvo/narrativeqa-test-raft"
test_dataset.push_to_hub(uploaded_dataset_id, split="test", private=True)
test_dataset_unique_ctx.push_to_hub(uploaded_dataset_id, split="test_unique_ctx", private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.34s/it]
c:\Users\vohun\anaconda3\envs\py10\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vohun\.cache\huggingface\hub\datasets--phatvo--narrativeqa-test-raft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/narrativeqa-test-raft/commit/b6e03590a6b21d495664c9217ecd0d9e8b18b04f', commit_message='Upload dataset', commit_description='', oid='b6e03590a6b21d495664c9217ecd0d9e8b18b04f', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
def save_context_to_file(data:list, filepath):
  with open(filepath, "w") as f:
    _data = [each.replace("\n", "") for each in data]
    f.write("\n".join(_data) + "\n")

save_context_to_file(raft_train_ctx, "../data/context_for_generate_narrativeqa.txt")

In [ ]:
"""

python3 raft.py \
    --datapath "data/context_for_generate_narrativeqa.txt" \
    --output "tmp/narrativeqa-test-50-raft2" \
    --distractors 3 \
    --doctype txt \
    --chunk_size 512 \
    --questions 1 \
    --openai_key $OPENAI_KEY \
    --completion_model gpt-4o \
    --splitter breakline --p 0.9

"""

In [ ]:
# $env:OPENAI_KEY=

In [1]:
from datasets import load_dataset

ds = load_dataset("json", data_files="../tmp/narrativeqa-test-50-raft2.jsonl", )
ds = ds.map(lambda x: {"text": f"{x['instruction']}\nCoT Answer: {x['cot_answer']}"})
ds

c:\Users\vohun\anaconda3\envs\py10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 50 examples [00:00, 2731.73 examples/s]
Map: 100%|██████████| 50/50 [00:00<00:00, 1438.10 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 50
    })
})

In [3]:
ds.push_to_hub("phatvo/narrativeqa-raft-50-p0.9")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.74s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/phatvo/narrativeqa-raft-50-p0.9/commit/19e7a364805766846a5783c5ae2ec55ef5b09077', commit_message='Upload dataset', commit_description='', oid='19e7a364805766846a5783c5ae2ec55ef5b09077', pr_url=None, pr_revision=None, pr_num=None)